In [3]:
%load_ext autoreload
%autoreload 2

import glob
import os

from IPython.display import display

import pandas as pd
import screed
import sourmash

base_folder = "/Users/olgabot/botryllus/data/botryllus-proteins"

gather_folder = os.path.join(base_folder, "sourmash_gather_scale10")
search_folder = os.path.join(base_folder, "sourmash_search_scale10")

## `parse_sourmash.py`

In [89]:
%%file parse_sourmash.py
import glob
import os

import pandas as pd


def read_csvs(folder, csv_glob='*.csv'):
    dfs = []
    csvs = glob.glob(os.path.join(folder, csv_glob))
    for csv in csvs:
        df = pd.read_csv(csv)
        dfs.append(df)
    concatenated = pd.concat(dfs, ignore_index=True)
    return concatenated




def extract_pfam_metadata(df):
    pfam_metadata_split1 = df.name.str.rstrip(";").str.split(";", expand=True)
    pfam_metadata_split2 = pfam_metadata_split1[0].str.split(expand=True)

    pfam_name = pfam_metadata_split1[1]
    pfam_name.name = "pfam_name"
    pfam_metadata = pd.concat([pfam_name, pfam_metadata_split2], axis=1)
    pfam_metadata = pfam_metadata.rename(columns={2: "pfam_id_full"})
    pfam_metadata["pfam_id"] = pfam_metadata.pfam_id_full.str.split(".").str[0]
    return pfam_metadata


def reorder_cols(df, first_cols=["similarity", "query_name"]):
    columns_reordered = first_cols + df.columns.difference(first_cols).tolist()
    df = df[columns_reordered]
    return df


def add_pfam_metadata(df, first_cols=["pfam_name", "similarity", "query_name"]):
    metadata = extract_pfam_metadata(df)
    df_with_metadata = pd.concat([df, metadata], axis=1)
    df_with_metadata = reorder_cols(
        df_with_metadata, first_cols
    )
    return df_with_metadata

def extract_gencode_gene_symbol(df, symbol_regex="gene_symbol:([\w\d\-]+)"):
    df["symbol"] = df["name"].str.extract(symbol_regex)
    return df

def read_gencode_folder(folder, csv_glob, first_cols, symbol_regex):
    df = read_csvs(folder, csv_glob)
    df = extract_gencode_gene_symbol(df, symbol_regex)
    df = reorder_cols(df, first_cols)
    return df

Overwriting parse_sourmash.py


## `kmer_utils`

In [53]:
%%file kmer_utils.py

import sourmash
from orpheum.sequence_encodings import encode_peptide


import pandas as pd

def get_encoded_kmer_hashvals(sequence, name, encoding="hp", K=24, sigobj=None):
    lines = []
    for i in range(0, len(sequence) - K + 1):
        kmer = sequence[i : i + K]
        kmer_encoded = encode_peptide(kmer, encoding)
        hashval = sigobj.minhash.seq_to_hashes(kmer, is_protein=True)

        if len(hashval) == 1:
            hashval = hashval[0]
        else:
            print(f"More than one hashval found for {kmer}")
        line = [i, kmer, kmer_encoded, hashval]
        lines.append(line)
    kmer_to_hashes = pd.DataFrame(
        lines, columns=["i", "kmer", f"kmer_{encoding}", "hashval"]
    )
    kmer_to_hashes["name"] = name
    return kmer_to_hashes

def get_matching_kmer_subsequence(sequence, kmers, gene_symbol):
    i_min = len(sequence)
    i_max = 0
    for kmer_human in kmers:
        i_kmer = sequence.find(kmer_human)
        # if i_kmer =< 0:
        #     continue

        if i_kmer < i_min:
            i_min = i_kmer
        if i_kmer > i_max:
            i_max = i_kmer
    j = i_max + len(kmer_human)

    #     print(f'i_min: {i_min}')
    #     print(f'i_max: {i_max}')
    #     print(f'j: {j}')

    
    print(f">{gene_symbol}")
    print(f"before match: {sequence[:i_min]}\n")
    print(f"matching: {sequence[i_min:j]}\n")
    print(f"after match: {sequence[j:]}")
    print(f"Match range (1-based): {i_min+1}-{j}")
    return sequence[i_min:j]


def subset_gene_kmers(merged_kmers, col, gene_symbol, other='human'):
    gene_subset = merged_kmers.query(f"{col} == @gene_symbol")
    
    kmer_other = f"kmer__{other}"

    tidy = pd.concat(
        [
            gene_subset[[kmer_other, "kmer_hp"]],
            gene_subset[["kmer__botryllus", "kmer_hp"]],
        ]
    )
    tidy = tidy.drop_duplicates()
    tidy["species"] = tidy[kmer_other].map(
        lambda x: "botryllus" if pd.isnull(x) else other
    )
    tidy["kmer_seq"] = tidy.apply(
        lambda x: x[kmer_other]
        if pd.isnull(x["kmer__botryllus"])
        else x["kmer__botryllus"],
        axis=1,
    )
    pivoted = tidy.pivot(columns="kmer_hp", index="species", values="kmer_seq")
    return pivoted

Overwriting kmer_utils.py


## Import `parse_sourmash`

In [21]:
import parse_sourmash
import kmer_utils


## Read gather output

In [8]:
gather = parse_sourmash.read_csvs(gather_folder)
print(gather.shape)
gather.head()

(14, 19)


,intersect_bp,f_orig_query,f_match,f_unique_to_query,f_unique_weighted,average_abund,median_abund,std_abund,name,filename,md5,f_match_orig,unique_intersect_bp,gather_result_rank,remaining_bp,query_filename,query_name,query_md5,query_bp
0,30,0.166667,0.031915,0.166667,0.166667,1.0,1.0,0.0,A0A6Q2YJA0_ESOLU/18-823 A0A6Q2YJA0.1 PF04147.1...,../pfam-split-seqkit-1e6/Pfam-A.prefetch.bhf_i...,5273d9b9abca82cb97efe29ee8d89149,0.031915,30,0,130,bhf_isoforms.fa,BHF_isoform1,f1f15760,180
1,30,0.166667,0.130435,0.166667,0.166667,1.0,1.0,0.0,A0A022QCL9_ERYGU/105-368 A0A022QCL9.1 PF03936....,../pfam-split-seqkit-1e6/Pfam-A.prefetch.bhf_i...,68fe401bebf694bd45c6491327b7bd76,0.130435,30,1,100,bhf_isoforms.fa,BHF_isoform1,f1f15760,180
2,30,0.166667,0.107143,0.166667,0.166667,1.0,1.0,0.0,A0A098M3F2_9BACL/57-334 A0A098M3F2.1 PF01297.2...,../pfam-split-seqkit-1e6/Pfam-A.prefetch.bhf_i...,031d2daae71f955d36562f75f30d8bea,0.107143,30,2,70,bhf_isoforms.fa,BHF_isoform1,f1f15760,180
3,20,0.111111,0.500000,0.111111,0.111111,1.0,1.0,0.0,A0A1Y2CQP8_9FUNG/32-76 A0A1Y2CQP8.1 PF00187.22...,../pfam-split-seqkit-1e6/Pfam-A.prefetch.bhf_i...,6f01987979bbf51f859d8816240127e9,0.500000,20,3,50,bhf_isoforms.fa,BHF_isoform1,f1f15760,180
4,20,0.111111,0.181818,0.111111,0.111111,1.0,1.0,0.0,B4HZG1_DROSE/1-124 B4HZG1.1 PF06382.14;Protami...,../pfam-split-seqkit-1e6/Pfam-A.prefetch.bhf_i...,35467a71a8e1a92a676557cad5575d43,0.181818,20,4,30,bhf_isoforms.fa,BHF_isoform1,f1f15760,180


In [9]:
gather_metadata = parse_sourmash.add_pfam_metadata(gather, first_cols=["pfam_name", "intersect_bp", "query_name"])
gather_metadata.head()

,pfam_name,intersect_bp,query_name,0,1,average_abund,f_match,f_match_orig,f_orig_query,f_unique_to_query,...,median_abund,name,pfam_id,pfam_id_full,query_bp,query_filename,query_md5,remaining_bp,std_abund,unique_intersect_bp
0,Nop14,30,BHF_isoform1,A0A6Q2YJA0_ESOLU/18-823,A0A6Q2YJA0.1,1.0,0.031915,0.031915,0.166667,0.166667,...,1.0,A0A6Q2YJA0_ESOLU/18-823 A0A6Q2YJA0.1 PF04147.1...,PF04147,PF04147.15,180,bhf_isoforms.fa,f1f15760,130,0.0,30
1,Terpene_synth_C,30,BHF_isoform1,A0A022QCL9_ERYGU/105-368,A0A022QCL9.1,1.0,0.130435,0.130435,0.166667,0.166667,...,1.0,A0A022QCL9_ERYGU/105-368 A0A022QCL9.1 PF03936....,PF03936,PF03936.19,180,bhf_isoforms.fa,f1f15760,100,0.0,30
2,ZnuA,30,BHF_isoform1,A0A098M3F2_9BACL/57-334,A0A098M3F2.1,1.0,0.107143,0.107143,0.166667,0.166667,...,1.0,A0A098M3F2_9BACL/57-334 A0A098M3F2.1 PF01297.2...,PF01297,PF01297.20,180,bhf_isoforms.fa,f1f15760,70,0.0,30
3,Chitin_bind_1,20,BHF_isoform1,A0A1Y2CQP8_9FUNG/32-76,A0A1Y2CQP8.1,1.0,0.500000,0.500000,0.111111,0.111111,...,1.0,A0A1Y2CQP8_9FUNG/32-76 A0A1Y2CQP8.1 PF00187.22...,PF00187,PF00187.22,180,bhf_isoforms.fa,f1f15760,50,0.0,20
4,Protamine_like,20,BHF_isoform1,B4HZG1_DROSE/1-124,B4HZG1.1,1.0,0.181818,0.181818,0.111111,0.111111,...,1.0,B4HZG1_DROSE/1-124 B4HZG1.1 PF06382.14;Protami...,PF06382,PF06382.14,180,bhf_isoforms.fa,f1f15760,30,0.0,20


In [44]:
gather_metadata

,pfam_name,intersect_bp,query_name,0,1,average_abund,f_match,f_match_orig,f_orig_query,f_unique_to_query,...,median_abund,name,pfam_id,pfam_id_full,query_bp,query_filename,query_md5,remaining_bp,std_abund,unique_intersect_bp
0,Nop14,30,BHF_isoform1,A0A6Q2YJA0_ESOLU/18-823,A0A6Q2YJA0.1,1.0,0.031915,0.031915,0.166667,0.166667,...,1.0,A0A6Q2YJA0_ESOLU/18-823 A0A6Q2YJA0.1 PF04147.1...,PF04147,PF04147.15,180,bhf_isoforms.fa,f1f15760,130,0.0,30
1,Terpene_synth_C,30,BHF_isoform1,A0A022QCL9_ERYGU/105-368,A0A022QCL9.1,1.0,0.130435,0.130435,0.166667,0.166667,...,1.0,A0A022QCL9_ERYGU/105-368 A0A022QCL9.1 PF03936....,PF03936,PF03936.19,180,bhf_isoforms.fa,f1f15760,100,0.0,30
2,ZnuA,30,BHF_isoform1,A0A098M3F2_9BACL/57-334,A0A098M3F2.1,1.0,0.107143,0.107143,0.166667,0.166667,...,1.0,A0A098M3F2_9BACL/57-334 A0A098M3F2.1 PF01297.2...,PF01297,PF01297.20,180,bhf_isoforms.fa,f1f15760,70,0.0,30
3,Chitin_bind_1,20,BHF_isoform1,A0A1Y2CQP8_9FUNG/32-76,A0A1Y2CQP8.1,1.0,0.500000,0.500000,0.111111,0.111111,...,1.0,A0A1Y2CQP8_9FUNG/32-76 A0A1Y2CQP8.1 PF00187.22...,PF00187,PF00187.22,180,bhf_isoforms.fa,f1f15760,50,0.0,20
4,Protamine_like,20,BHF_isoform1,B4HZG1_DROSE/1-124,B4HZG1.1,1.0,0.181818,0.181818,0.111111,0.111111,...,1.0,B4HZG1_DROSE/1-124 B4HZG1.1 PF06382.14;Protami...,PF06382,PF06382.14,180,bhf_isoforms.fa,f1f15760,30,0.0,20
5,2-Hacid_dh,10,BHF_isoform1,A0A3P3D7T7_9RHOB/8-324,A0A3P3D7T7.1,1.0,0.030303,0.030303,0.055556,0.055556,...,1.0,A0A3P3D7T7_9RHOB/8-324 A0A3P3D7T7.1 PF00389.33...,PF00389,PF00389.33,180,bhf_isoforms.fa,f1f15760,20,0.0,10
6,4HB_MCP_1,10,BHF_isoform1,R5Z925_9FIRM/1-181,R5Z925.1,1.0,0.058824,0.058824,0.055556,0.055556,...,1.0,R5Z925_9FIRM/1-181 R5Z925.1 PF12729.10;4HB_MCP_1;,PF12729,PF12729.10,180,bhf_isoforms.fa,f1f15760,10,0.0,10
7,7TM_GPCR_Srh,10,BHF_isoform1,E3LLJ4_CAERE/4-302,E3LLJ4.1,1.0,0.043478,0.043478,0.055556,0.055556,...,1.0,E3LLJ4_CAERE/4-302 E3LLJ4.1 PF10318.12;7TM_GPC...,PF10318,PF10318.12,180,bhf_isoforms.fa,f1f15760,0,0.0,10
8,Nop14,30,BHF_isoform2,A0A6Q2YJA0_ESOLU/18-823,A0A6Q2YJA0.1,1.0,0.031915,0.031915,0.187500,0.187500,...,1.0,A0A6Q2YJA0_ESOLU/18-823 A0A6Q2YJA0.1 PF04147.1...,PF04147,PF04147.15,160,bhf_isoforms.fa,865a91c0,110,0.0,30
9,Terpene_synth_C,30,BHF_isoform2,A0A022QCL9_ERYGU/105-368,A0A022QCL9.1,1.0,0.130435,0.130435,0.187500,0.187500,...,1.0,A0A022QCL9_ERYGU/105-368 A0A022QCL9.1 PF03936....,PF03936,PF03936.19,160,bhf_isoforms.fa,865a91c0,80,0.0,30


### Add metadata

## Read search output

In [10]:
search = parse_sourmash.read_csvs(search_folder)
print(search.shape)
search.head()

(4728, 7)


,similarity,name,filename,md5,query_filename,query_name,query_md5
0,0.230053,A0A5N5NYA3_PANHP/704-740 A0A5N5NYA3.1 PF00057....,NaN,8236a3e260acf15aa3cc673821003cb7,bhf_isoforms.fa,BHF_isoform2,865a91c0
1,0.186429,A0A6J2X2G5_SITOR/131-211 A0A6J2X2G5.1 PF18391....,NaN,cb641ff362c34d379be11a7302674fc4,bhf_isoforms.fa,BHF_isoform2,865a91c0
2,0.186429,H0Y978_HUMAN/30-85 H0Y978.1 PF00400.35;WD40;,NaN,41dd6480cafa2c71272010815ad0ec96,bhf_isoforms.fa,BHF_isoform2,865a91c0
3,0.186429,A0A673I3J8_9TELE/6-59 A0A673I3J8.1 PF00105.21;...,NaN,15cbdeede334ae99e2e3443133898932,bhf_isoforms.fa,BHF_isoform2,865a91c0
4,0.160861,A0A6G0XDQ6_9STRA/342-370 A0A6G0XDQ6.1 PF00734....,NaN,1a401620b607e1f0e1c3ae66fbe5e5c3,bhf_isoforms.fa,BHF_isoform2,865a91c0


### Add metadata

In [11]:


search_metadata = parse_sourmash.add_pfam_metadata(search)
search_metadata.head()

,pfam_name,similarity,query_name,0,1,filename,md5,name,pfam_id,pfam_id_full,query_filename,query_md5
0,Ldl_recept_a,0.230053,BHF_isoform2,A0A5N5NYA3_PANHP/704-740,A0A5N5NYA3.1,NaN,8236a3e260acf15aa3cc673821003cb7,A0A5N5NYA3_PANHP/704-740 A0A5N5NYA3.1 PF00057....,PF00057,PF00057.21,bhf_isoforms.fa,865a91c0
1,CHIP_TPR_N,0.186429,BHF_isoform2,A0A6J2X2G5_SITOR/131-211,A0A6J2X2G5.1,NaN,cb641ff362c34d379be11a7302674fc4,A0A6J2X2G5_SITOR/131-211 A0A6J2X2G5.1 PF18391....,PF18391,PF18391.4,bhf_isoforms.fa,865a91c0
2,WD40,0.186429,BHF_isoform2,H0Y978_HUMAN/30-85,H0Y978.1,NaN,41dd6480cafa2c71272010815ad0ec96,H0Y978_HUMAN/30-85 H0Y978.1 PF00400.35;WD40;,PF00400,PF00400.35,bhf_isoforms.fa,865a91c0
3,zf-C4,0.186429,BHF_isoform2,A0A673I3J8_9TELE/6-59,A0A673I3J8.1,NaN,15cbdeede334ae99e2e3443133898932,A0A673I3J8_9TELE/6-59 A0A673I3J8.1 PF00105.21;...,PF00105,PF00105.21,bhf_isoforms.fa,865a91c0
4,CBM_1,0.160861,BHF_isoform2,A0A6G0XDQ6_9STRA/342-370,A0A6G0XDQ6.1,NaN,1a401620b607e1f0e1c3ae66fbe5e5c3,A0A6G0XDQ6_9STRA/342-370 A0A6G0XDQ6.1 PF00734....,PF00734,PF00734.21,bhf_isoforms.fa,865a91c0


# Get K-mers

## BHF

### File paths

In [13]:

bhf_isoform_sigfile = '/Users/olgabot/botryllus/data/botryllus-proteins/bhf_isoforms.fa.sig'
bhf_isoform_fasta = '/Users/olgabot/botryllus/data/botryllus-proteins/bhf_isoforms.fa'


### Read in BHF isoform fasta

In [18]:
bhf_isoform_seqs = {record.name: record.sequence for record in screed.open(bhf_isoform_fasta)}
bhf_isoform_seqs

{'BHF_isoform1': 'MVHDTEQLLAQGHHEEETECGKYGKLPKKGSECKKHGILCRILTALHLKKRRTKHDHQKLLSESQEHIDASTNKTKKKAKKDKRKNKPPKKDSETSKPAQTTISRLPSNRNNNNANSFATTYEKFDNDSLCSVDLIPVDIEFWDMENEPVDQLPHEIXESVHMYGDDRFGERLIDRAQNKYAPLDEKQRSESHGAGEYLKHQWKGQGAKKARKRIRTVMKATWQSLQAGARSQTAFLNPQGAVSAALVQNRR',
 'BHF_isoform2': 'MVHDTEQLLAQGHHEEETECGKYGKLPKKGSECKKHGILCRILTALHLKKRRTKHDHQKLLSESQEHIDASTNKTKKKAKKDKRKNKPPKKDSETSKPAQTTISRLPSNRNNNNANSFATTYEKFDNDSLCSVDLIPVDIEFWDMENEPVDQLPHEIXESVHMYGDDRFGERLIDRAQNKYAPLDEKQRSESHGAGEYLKHQWKGQGAKKARKVRSNGV'}

### Read in the botryllus sigfile to have a minhash device

In [16]:

bhf_isoform_sigs = {sig.name: sig for sig in sourmash.load_file_as_signatures(bhf_isoform_sigfile)}
bhf_isoform_sigs

{'BHF_isoform1': SourmashSignature('BHF_isoform1', f1f15760),
 'BHF_isoform2': SourmashSignature('BHF_isoform2', 865a91c0)}

### Get BHF K-mers

In [36]:
dfs = []
for isoform, seq in bhf_isoform_seqs.items():
    sigobj = bhf_isoform_sigs[isoform]
    df = kmer_utils.get_encoded_kmer_hashvals(seq, isoform, sigobj=sigobj)
    dfs.append(df)
bhf_kmers = pd.concat(dfs, ignore_index=True)
bhf_kmers.head()

,i,kmer,kmer_hp,hashval,name
0,0,MVHDTEQLLAQGHHEEETECGKYG,hhppppphhhphpppppppphphh,17134648382419275520,BHF_isoform1
1,1,VHDTEQLLAQGHHEEETECGKYGK,hppppphhhphpppppppphphhp,1036020595944595459,BHF_isoform1
2,2,HDTEQLLAQGHHEEETECGKYGKL,ppppphhhphpppppppphphhph,9354784992242920062,BHF_isoform1
3,3,DTEQLLAQGHHEEETECGKYGKLP,pppphhhphpppppppphphhphh,14467040310155683947,BHF_isoform1
4,4,TEQLLAQGHHEEETECGKYGKLPK,ppphhhphpppppppphphhphhp,2993707203445337902,BHF_isoform1


## PFam

In [31]:
pfam_fasta = '/Users/olgabot/botryllus/data/pfam/Pfam-A.prefetch.bhf_isoforms.threshold0.fasta.gz'
pfam_sigfile = '/Users/olgabot/botryllus/data/pfam/scaled10/Pfam-A.prefetch.bhf_isoforms.threshold0.sig.gz'

In [32]:
pfam_seqs = {record.name: record.sequence for record in screed.open(pfam_fasta)}

### Get Pfam kmers

In [34]:
# use last sigobj
dfs = []
for isoform, seq in pfam_seqs.items():
    df = kmer_utils.get_encoded_kmer_hashvals(seq, isoform, sigobj=sigobj)
    dfs.append(df)
pfam_kmers = pd.concat(dfs, ignore_index=True)
pfam_kmers.head()

,i,kmer,kmer_hp,hashval,name
0,0,LLTLNAISVKGLDRLPRDSYEIAS,hhphphhphphhpphhppphphhp,12419877632473437588,F3L153_9GAMM/5-309 F3L153.1 PF00389.33;2-Hacid...
1,1,LTLNAISVKGLDRLPRDSYEIASE,hphphhphphhpphhppphphhpp,5304522194303411330,F3L153_9GAMM/5-309 F3L153.1 PF00389.33;2-Hacid...
2,2,TLNAISVKGLDRLPRDSYEIASEF,phphhphphhpphhppphphhpph,1326463173270241912,F3L153_9GAMM/5-309 F3L153.1 PF00389.33;2-Hacid...
3,3,LNAISVKGLDRLPRDSYEIASEFS,hphhphphhpphhppphphhpphp,5050722563078200312,F3L153_9GAMM/5-309 F3L153.1 PF00389.33;2-Hacid...
4,4,NAISVKGLDRLPRDSYEIASEFSA,phhphphhpphhppphphhpphph,18176231415180590896,F3L153_9GAMM/5-309 F3L153.1 PF00389.33;2-Hacid...


In [39]:
pfam_kmers.shape

(600426, 5)

In [38]:
pfam_kmers_metdata = parse_sourmash.add_pfam_metadata(pfam_kmers, first_cols=['pfam_name', 'kmer', 'kmer_hp'])
pfam_kmers_metdata.head()

,pfam_name,kmer,kmer_hp,0,1,hashval,i,name,pfam_id,pfam_id_full
0,2-Hacid_dh,LLTLNAISVKGLDRLPRDSYEIAS,hhphphhphphhpphhppphphhp,F3L153_9GAMM/5-309,F3L153.1,12419877632473437588,0,F3L153_9GAMM/5-309 F3L153.1 PF00389.33;2-Hacid...,PF00389,PF00389.33
1,2-Hacid_dh,LTLNAISVKGLDRLPRDSYEIASE,hphphhphphhpphhppphphhpp,F3L153_9GAMM/5-309,F3L153.1,5304522194303411330,1,F3L153_9GAMM/5-309 F3L153.1 PF00389.33;2-Hacid...,PF00389,PF00389.33
2,2-Hacid_dh,TLNAISVKGLDRLPRDSYEIASEF,phphhphphhpphhppphphhpph,F3L153_9GAMM/5-309,F3L153.1,1326463173270241912,2,F3L153_9GAMM/5-309 F3L153.1 PF00389.33;2-Hacid...,PF00389,PF00389.33
3,2-Hacid_dh,LNAISVKGLDRLPRDSYEIASEFS,hphhphphhpphhppphphhpphp,F3L153_9GAMM/5-309,F3L153.1,5050722563078200312,3,F3L153_9GAMM/5-309 F3L153.1 PF00389.33;2-Hacid...,PF00389,PF00389.33
4,2-Hacid_dh,NAISVKGLDRLPRDSYEIASEFSA,phhphphhpphhppphphhpphph,F3L153_9GAMM/5-309,F3L153.1,18176231415180590896,4,F3L153_9GAMM/5-309 F3L153.1 PF00389.33;2-Hacid...,PF00389,PF00389.33


## Merge BHF and pfam kmers

In [48]:
bhf_pfam_kmers = pfam_kmers_metdata.merge(
    bhf_kmers,
    how="inner",
    on=["hashval", "kmer_hp"],
    suffixes=("__pfam", "__botryllus"),
)
print(bhf_pfam_kmers.shape)
bhf_pfam_kmers.head()

(14829, 13)


,pfam_name,kmer__pfam,kmer_hp,0,1,hashval,i__pfam,name__pfam,pfam_id,pfam_id_full,i__botryllus,kmer__botryllus,name__botryllus
0,2-Hacid_dh,VEGVQWVNTLTDHNDKAALNTLLE,hphhphhpphpppppphhhpphhp,F3L153_9GAMM/5-309,F3L153.1,6861319682135420868,99,F3L153_9GAMM/5-309 F3L153.1 PF00389.33;2-Hacid...,PF00389,PF00389.33,20,GKYGKLPKKGSECKKHGILCRILT,BHF_isoform1
1,2-Hacid_dh,VEGVQWVNTLTDHNDKAALNTLLE,hphhphhpphpppppphhhpphhp,F3L153_9GAMM/5-309,F3L153.1,6861319682135420868,99,F3L153_9GAMM/5-309 F3L153.1 PF00389.33;2-Hacid...,PF00389,PF00389.33,20,GKYGKLPKKGSECKKHGILCRILT,BHF_isoform2
2,2-Hacid_dh_C,VEGVQWVNTLTDHNDKAALNTLLE,hphhphhpphpppppphhhpphhp,F3L153_9GAMM/98-277,F3L153.1,6861319682135420868,6,F3L153_9GAMM/98-277 F3L153.1 PF02826.22;2-Haci...,PF02826,PF02826.22,20,GKYGKLPKKGSECKKHGILCRILT,BHF_isoform1
3,2-Hacid_dh_C,VEGVQWVNTLTDHNDKAALNTLLE,hphhphhpphpppppphhhpphhp,F3L153_9GAMM/98-277,F3L153.1,6861319682135420868,6,F3L153_9GAMM/98-277 F3L153.1 PF02826.22;2-Haci...,PF02826,PF02826.22,20,GKYGKLPKKGSECKKHGILCRILT,BHF_isoform2
4,AAA_10,INPLSFIDSINETNRKLGISKFVD,hphhphhpphpppppphhhpphhp,F4L4C0_HALH1/172-500,F4L4C0.1,6861319682135420868,80,F4L4C0_HALH1/172-500 F4L4C0.1 PF12846.10;AAA_10;,PF12846,PF12846.10,20,GKYGKLPKKGSECKKHGILCRILT,BHF_isoform1


In [59]:
bhf_pfam_kmers_isoform1 = bhf_pfam_kmers.query('name__botryllus == "BHF_isoform1"')
print(bhf_pfam_kmers_isoform1.shape)
bhf_pfam_kmers_isoform2 = bhf_pfam_kmers.query('name__botryllus == "BHF_isoform2"')
print(bhf_pfam_kmers_isoform2.shape)

(8194, 13)
(6635, 13)


In [62]:
bhf_pfam_kmers_isoform1_nop14 = bhf_pfam_kmers_isoform1.query('pfam_name == "Nop14"')
bhf_pfam_kmers_isoform1_nop14.shape

(34, 13)

In [63]:
bhf_pfam_kmers_isoform1_nop14.groupby('name__pfam').size()

name__pfam
A0A087HGT1_ARAAL/28-893 A0A087HGT1.1 PF04147.15;Nop14;     5
A0A3P8ZQ40_ESOLU/19-861 A0A3P8ZQ40.1 PF04147.15;Nop14;    12
A0A6J1KKH7_CUCMA/35-930 A0A6J1KKH7.1 PF04147.15;Nop14;     3
A0A6Q2YJA0_ESOLU/18-823 A0A6Q2YJA0.1 PF04147.15;Nop14;    12
M1VWC2_CLAP2/49-896 M1VWC2.1 PF04147.15;Nop14;             2
dtype: int64

In [67]:
for name, df in bhf_pfam_kmers_isoform1_nop14.groupby('name__pfam'):
    pivoted = kmer_utils.subset_gene_kmers(
        df, col="pfam_name", gene_symbol="Nop14", other="pfam"
    )
    print(f'--- {name} ---')
    display(pivoted)

--- A0A087HGT1_ARAAL/28-893 A0A087HGT1.1 PF04147.15;Nop14; ---


kmer_hp,pppphpppppppphhpppppppph,ppppphpppppppphhpppppppp,pppppphpppppppphhppppppp,ppppppphpppppppphhpppppp,pppppppphpppppppphhppppp
species,,,,,
botryllus,TKKKAKKDKRKNKPPKKDSETSKP,KTKKKAKKDKRKNKPPKKDSETSK,NKTKKKAKKDKRKNKPPKKDSETS,TNKTKKKAKKDKRKNKPPKKDSET,STNKTKKKAKKDKRKNKPPKKDSE
pfam,DEDDGSSEDSESEGGEDDDDDDDG,SDEDDGSSEDSESEGGEDDDDDDD,ESDEDDGSSEDSESEGGEDDDDDD,SESDEDDGSSEDSESEGGEDDDDD,NSESDEDDGSSEDSESEGGEDDDD


--- A0A3P8ZQ40_ESOLU/19-861 A0A3P8ZQ40.1 PF04147.15;Nop14; ---


kmer_hp,hphpppppppphpppppppphhpp,hpppppppphpppppppphhpppp,phphpppppppphpppppppphhp,phpppppppphpppppppphhppp,pphphpppppppphpppppppphh,ppphphpppppppphpppppppph,ppphpppppppphhpppppppphh,pppphpppppppphhpppppppph,ppppphpppppppphhpppppppp,pppppphpppppppphhppppppp,ppppppphpppppppphhpppppp,pppppppphpppppppphhppppp
species,,,,,,,,,,,,
botryllus,IDASTNKTKKKAKKDKRKNKPPKK,ASTNKTKKKAKKDKRKNKPPKKDS,HIDASTNKTKKKAKKDKRKNKPPK,DASTNKTKKKAKKDKRKNKPPKKD,EHIDASTNKTKKKAKKDKRKNKPP,QEHIDASTNKTKKKAKKDKRKNKP,KKKAKKDKRKNKPPKKDSETSKPA,TKKKAKKDKRKNKPPKKDSETSKP,KTKKKAKKDKRKNKPPKKDSETSK,NKTKKKAKKDKRKNKPPKKDSETS,TNKTKKKAKKDKRKNKPPKKDSET,STNKTKKKAKKDKRKNKPPKKDSE
pfam,GEGSEEDSHSDVNSEEESEEGGEE,GSEEDSHSDVNSEEESEEGGEEEE,DGEGSEEDSHSDVNSEEESEEGGE,EGSEEDSHSDVNSEEESEEGGEEE,EDGEGSEEDSHSDVNSEEESEEGG,EEDGEGSEEDSHSDVNSEEESEEG,HSDVNSEEESEEGGEEEEEDEQPA,SHSDVNSEEESEEGGEEEEEDEQP,DSHSDVNSEEESEEGGEEEEEDEQ,EDSHSDVNSEEESEEGGEEEEEDE,EEDSHSDVNSEEESEEGGEEEEED,SEEDSHSDVNSEEESEEGGEEEEE


--- A0A6J1KKH7_CUCMA/35-930 A0A6J1KKH7.1 PF04147.15;Nop14; ---


kmer_hp,pphphpppppppphpppppppphh,ppphphpppppppphpppppppph,pppphphpppppppphpppppppp
species,,,
botryllus,EHIDASTNKTKKKAKKDKRKNKPP,QEHIDASTNKTKKKAKKDKRKNKP,SQEHIDASTNKTKKKAKKDKRKNK
pfam,KDADGTESEDDDSAEDTDSSDDVG,RKDADGTESEDDDSAEDTDSSDDV,ERKDADGTESEDDDSAEDTDSSDD


--- A0A6Q2YJA0_ESOLU/18-823 A0A6Q2YJA0.1 PF04147.15;Nop14; ---


kmer_hp,hphpppppppphpppppppphhpp,hpppppppphpppppppphhpppp,phphpppppppphpppppppphhp,phpppppppphpppppppphhppp,pphphpppppppphpppppppphh,ppphphpppppppphpppppppph,ppphpppppppphhpppppppphh,pppphpppppppphhpppppppph,ppppphpppppppphhpppppppp,pppppphpppppppphhppppppp,ppppppphpppppppphhpppppp,pppppppphpppppppphhppppp
species,,,,,,,,,,,,
botryllus,IDASTNKTKKKAKKDKRKNKPPKK,ASTNKTKKKAKKDKRKNKPPKKDS,HIDASTNKTKKKAKKDKRKNKPPK,DASTNKTKKKAKKDKRKNKPPKKD,EHIDASTNKTKKKAKKDKRKNKPP,QEHIDASTNKTKKKAKKDKRKNKP,KKKAKKDKRKNKPPKKDSETSKPA,TKKKAKKDKRKNKPPKKDSETSKP,KTKKKAKKDKRKNKPPKKDSETSK,NKTKKKAKKDKRKNKPPKKDSETS,TNKTKKKAKKDKRKNKPPKKDSET,STNKTKKKAKKDKRKNKPPKKDSE
pfam,GEGSEEDSHSDVNSEEESEEGGEE,GSEEDSHSDVNSEEESEEGGEEEE,DGEGSEEDSHSDVNSEEESEEGGE,EGSEEDSHSDVNSEEESEEGGEEE,EDGEGSEEDSHSDVNSEEESEEGG,EEDGEGSEEDSHSDVNSEEESEEG,HSDVNSEEESEEGGEEEEEDEQPA,SHSDVNSEEESEEGGEEEEEDEQP,DSHSDVNSEEESEEGGEEEEEDEQ,EDSHSDVNSEEESEEGGEEEEEDE,EEDSHSDVNSEEESEEGGEEEEED,SEEDSHSDVNSEEESEEGGEEEEE


--- M1VWC2_CLAP2/49-896 M1VWC2.1 PF04147.15;Nop14; ---


kmer_hp,ppphphpppppppphpppppppph,pppphphpppppppphpppppppp
species,,
botryllus,QEHIDASTNKTKKKAKKDKRKNKP,SQEHIDASTNKTKKKAKKDKRKNK
pfam,SDEGSYDDDSDNSQAEEDEEDDEF,ESDEGSYDDDSDNSQAEEDEEDDE


## Iterate over gather hits:


In [83]:
for isoform, df in gather_metadata.groupby("query_name"):
    print(f"\n--- {isoform} ---")
    for i, row in df.iterrows():
        pfam_full = row["name"]
        intersect_bp = row['intersect_bp']
        pivoted = kmer_utils.subset_gene_kmers(
            bhf_pfam_kmers, col="name__pfam", gene_symbol=pfam_full, other="pfam"
        )
        print(f"\n\n--- {pfam_full}, intersect_bp: {intersect_bp} ---")
        display(pivoted)

        kmer_utils.get_matching_kmer_subsequence(
            pfam_seqs[pfam_full], pivoted.loc["pfam"], pfam_full
        )
        print("-")
        kmer_utils.get_matching_kmer_subsequence(
            bhf_isoform_seqs[isoform], pivoted.loc["botryllus"], "BHF"
        )


--- BHF_isoform1 ---


--- A0A6Q2YJA0_ESOLU/18-823 A0A6Q2YJA0.1 PF04147.15;Nop14;, intersect_bp: 30 ---


kmer_hp,hphpppppppphpppppppphhpp,hpppppppphpppppppphhpppp,phphpppppppphpppppppphhp,phpppppppphpppppppphhppp,pphphpppppppphpppppppphh,ppphphpppppppphpppppppph,ppphpppppppphhpppppppphh,pppphpppppppphhpppppppph,ppppphpppppppphhpppppppp,pppppphpppppppphhppppppp,ppppppphpppppppphhpppppp,pppppppphpppppppphhppppp
species,,,,,,,,,,,,
botryllus,IDASTNKTKKKAKKDKRKNKPPKK,ASTNKTKKKAKKDKRKNKPPKKDS,HIDASTNKTKKKAKKDKRKNKPPK,DASTNKTKKKAKKDKRKNKPPKKD,EHIDASTNKTKKKAKKDKRKNKPP,QEHIDASTNKTKKKAKKDKRKNKP,KKKAKKDKRKNKPPKKDSETSKPA,TKKKAKKDKRKNKPPKKDSETSKP,KTKKKAKKDKRKNKPPKKDSETSK,NKTKKKAKKDKRKNKPPKKDSETS,TNKTKKKAKKDKRKNKPPKKDSET,STNKTKKKAKKDKRKNKPPKKDSE
pfam,GEGSEEDSHSDVNSEEESEEGGEE,GSEEDSHSDVNSEEESEEGGEEEE,DGEGSEEDSHSDVNSEEESEEGGE,EGSEEDSHSDVNSEEESEEGGEEE,EDGEGSEEDSHSDVNSEEESEEGG,EEDGEGSEEDSHSDVNSEEESEEG,HSDVNSEEESEEGGEEEEEDEQPA,SHSDVNSEEESEEGGEEEEEDEQP,DSHSDVNSEEESEEGGEEEEEDEQ,EDSHSDVNSEEESEEGGEEEEEDE,EEDSHSDVNSEEESEEGGEEEEED,SEEDSHSDVNSEEESEEGGEEEEE


>A0A6Q2YJA0_ESOLU/18-823 A0A6Q2YJA0.1 PF04147.15;Nop14;
before match: KTSKEIKNNPFEVKINRKKFDILGRKSKHDVGLPGVSRSKAINKRKETLLKEYKTKDKSNKFIDKRFGEYDTKMDPEEKILQRFSMERQRTQDKKNMYNLNEEEELTHYGQSLAEMEKLTDMVDSDDDADERGLLSAEMTASHFGGGGLLMKKTPGEQDEGGKQRAKSRQELIEELIIKSKQEKRERQTQKEESQVLTEKLDQDWKSIQGLLAHKNAPKADRAEDEDKPKLDEYDMMVRELGFEMKAAPSEKLKTPEEVAREEQERLQQLEADRLRRMMGDIVEDNTKAPTHMSADDINDGFVLDGDDRQTLAYQVRDNINRLFTEGEKIEGEEGEKEESGEEEESDEEESEQEG

matching: EEDGEGSEEDSHSDVNSEEESEEGGEEEEEDEQPA

after match: AVPQLSEKVQAEAAKIELPYTFTVPECYRDLKSLLQGHPADHQCIILARTQQCNHPSLGIGNKLKIQVCYLIDIVCIDLLQLSNIFILYDLCQLFPETASKGLQTTLGDDAHSMEEVLEVKGRAAFPKLDMLIYLKITALLFPTSDFRHPVTTPALLYISQALTKCPVTSLKDLTSGLVLCCLALEYVSFSKRFVPELINFLLGALHLAVTDKTTGYSVVPPFRRSGKASDLLVLSSPESSQTWCKKTLPLSATQTLTLTSDLDSHMKLSILATCLDLLKRCFSLYRELPSCLHIFQPIATLLSKHLPIKTYPAALQVSGPALVSRKKKPIPLKLFTPRIVQVLDYGKKRGNTKEEREKERLKHKYKKEFKGALREIRKDSRFLGREKLNEVMARDAERKRKVKELYGSLATQEGE
Match range (1-based): 356-390
-
>BHF
before match: MVHDTEQLLAQGHHEEETECGKYGKLPKKGSECKKHGILCRILT

kmer_hp,hhphhpphpppppphhhpphhphh,hhpphpppppphhhpphhphhphp,hphhphhpphpppppphhhpphhp,hphhpphpppppphhhpphhphhp,hpphpppppphhhpphhphhphpp,hpppppppphphhphhpphppppp,phhphhpphpppppphhhpphhph,phhpphpppppphhhpphhphhph,phphhphhpphpppppphhhpphh,phpppppphhhpphhphhphpppp,phpppppppphphhphhpphpppp,pphphhphhpphpppppphhhpph,pphpppppphhhpphhphhphppp,ppphphhphhpphpppppphhhpp,pppphphhphhpphpppppphhhp,ppppphphhphhpphpppppphhh,pppppphphhphhpphpppppphh,ppppppphphhphhpphpppppph,pppppppphphhphhpphpppppp
species,,,,,,,,,,,,,,,,,,,
botryllus,YGKLPKKGSECKKHGILCRILTAL,LPKKGSECKKHGILCRILTALHLK,GKYGKLPKKGSECKKHGILCRILT,GKLPKKGSECKKHGILCRILTALH,PKKGSECKKHGILCRILTALHLKK,GHHEEETECGKYGKLPKKGSECKK,KYGKLPKKGSECKKHGILCRILTA,KLPKKGSECKKHGILCRILTALHL,CGKYGKLPKKGSECKKHGILCRIL,KGSECKKHGILCRILTALHLKKRR,QGHHEEETECGKYGKLPKKGSECK,ECGKYGKLPKKGSECKKHGILCRI,KKGSECKKHGILCRILTALHLKKR,TECGKYGKLPKKGSECKKHGILCR,ETECGKYGKLPKKGSECKKHGILC,EETECGKYGKLPKKGSECKKHGIL,EEETECGKYGKLPKKGSECKKHGI,HEEETECGKYGKLPKKGSECKKHG,HHEEETECGKYGKLPKKGSECKKH
pfam,AVSILNEARDKDEEMVVSEIGKLV,ILNEARDKDEEMVVSEIGKLVEYN,LNAVSILNEARDKDEEMVVSEIGK,VSILNEARDKDEEMVVSEIGKLVE,LNEARDKDEEMVVSEIGKLVEYNR,FQKEKEEKKLNAVSILNEARDKDE,NAVSILNEARDKDEEMVVSEIGKL,SILNEARDKDEEMVVSEIGKLVEY,KLNAVSILNEARDKDEEMVVSEIG,EARDKDEEMVVSEIGKLVEYNRRK,SFQKEKEEKKLNAVSILNEARDKD,KKLNAVSILNEARDKDEEMVVSEI,NEARDKDEEMVVSEIGKLVEYNRR,EKKLNAVSILNEARDKDEEMVVSE,EEKKLNAVSILNEARDKDEEMVVS,KEEKKLNAVSILNEARDKDEEMVV,EKEEKKLNAVSILNEARDKDEEMV,KEKEEKKLNAVSILNEARDKDEEM,QKEKEEKKLNAVSILNEARDKDEE


>A0A022QCL9_ERYGU/105-368 A0A022QCL9.1 PF03936.19;Terpene_synth_C;
before match: LVFARQDFNHCQALHQKEFQQLERWYKDCGLDRLNYGRNVLHVAHFIASTVIGDPQLVDARMSYAKHVVLVTRIDDFFDHHGSREESHKILELVKEWKEKPAVDYGSQEVEILFSAVYRTVNELAEKASVEQGRCVKHHLINLWVQILTSFVRELDTWCDDAAMTLDEYLSFAWVSIGCRICILTSIHFMGIRLSDDMILGEECTNLCIHVSTVNRLLNDLQ

matching: SFQKEKEEKKLNAVSILNEARDKDEEMVVSEIGKLVEYNRRK

after match: 
Match range (1-based): 223-264
-
>BHF
before match: MVHDTEQLLA

matching: QGHHEEETECGKYGKLPKKGSECKKHGILCRILTALHLKKRR

after match: TKHDHQKLLSESQEHIDASTNKTKKKAKKDKRKNKPPKKDSETSKPAQTTISRLPSNRNNNNANSFATTYEKFDNDSLCSVDLIPVDIEFWDMENEPVDQLPHEIXESVHMYGDDRFGERLIDRAQNKYAPLDEKQRSESHGAGEYLKHQWKGQGAKKARKRIRTVMKATWQSLQAGARSQTAFLNPQGAVSAALVQNRR
Match range (1-based): 11-52


--- A0A098M3F2_9BACL/57-334 A0A098M3F2.1 PF01297.20;ZnuA;, intersect_bp: 30 ---


...

>A0A1Y2CQP8_9FUNG/32-76 A0A1Y2CQP8.1 PF00187.22;Chitin_bind_1;
before match: SKDGRCGPNFGICPS

matching: GQCCSKYGYCGTSSDHCSKYCIPEYGDC

after match: SN
Match range (1-based): 16-43
-
>BHF
before match: M

matching: VHDTEQLLAQGHHEEETECGKYGKLPKK

after match: GSECKKHGILCRILTALHLKKRRTKHDHQKLLSESQEHIDASTNKTKKKAKKDKRKNKPPKKDSETSKPAQTTISRLPSNRNNNNANSFATTYEKFDNDSLCSVDLIPVDIEFWDMENEPVDQLPHEIXESVHMYGDDRFGERLIDRAQNKYAPLDEKQRSESHGAGEYLKHQWKGQGAKKARKVRSNGV
Match range (1-based): 2-29


--- B4HZG1_DROSE/1-124 B4HZG1.1 PF06382.14;Protamine_like;, intersect_bp: 20 ---


kmer_hp,hhpphhpphppphhhhpppppppp,hhpphppphhhhppppppppphhh,hpphhpphppphhhhppppppppp,hpphppphhhhppppppppphhhp,phhpphppphhhhppppppppphh,pphhpphppphhhhppppppppph
species,,,,,,
botryllus,FGERLIDRAQNKYAPLDEKQRSES,LIDRAQNKYAPLDEKQRSESHGAG,GERLIDRAQNKYAPLDEKQRSESH,IDRAQNKYAPLDEKQRSESHGAGE,RLIDRAQNKYAPLDEKQRSESHGA,ERLIDRAQNKYAPLDEKQRSESHG
pfam,PAQKVACDLKSDAAGGQQRSCQRQ,VACDLKSDAAGGQQRSCQRQSPYA,AQKVACDLKSDAAGGQQRSCQRQS,ACDLKSDAAGGQQRSCQRQSPYAR,KVACDLKSDAAGGQQRSCQRQSPY,QKVACDLKSDAAGGQQRSCQRQSP


>B4HZG1_DROSE/1-124 B4HZG1.1 PF06382.14;Protamine_like;
before match: MGQKRHRTYCPPTYKRQKVARITNNGYLNFMTEYKKRFYGLSPQDMVHYAAKQWTQLSSAEKEAFKSKKPPTVVFKG

matching: PAQKVACDLKSDAAGGQQRSCQRQSPYAR

after match: SRESERRLSRSKTSCKSA
Match range (1-based): 78-106
-
>BHF
before match: MVHDTEQLLAQGHHEEETECGKYGKLPKKGSECKKHGILCRILTALHLKKRRTKHDHQKLLSESQEHIDASTNKTKKKAKKDKRKNKPPKKDSETSKPAQTTISRLPSNRNNNNANSFATTYEKFDNDSLCSVDLIPVDIEFWDMENEPVDQLPHEIXESVHMYGDDR

matching: FGERLIDRAQNKYAPLDEKQRSESHGAGE

after match: YLKHQWKGQGAKKARKVRSNGV
Match range (1-based): 169-197


--- E3LLJ4_CAERE/4-302 E3LLJ4.1 PF10318.12;7TM_GPCR_Srh;, intersect_bp: 10 ---


kmer_hp,hhpphpphpppphpphphhhhphp,hpphpphpppphpphphhhhphph,pphpphpppphpphphhhhphphh
species,,,
botryllus,FATTYEKFDNDSLCSVDLIPVDIE,ATTYEKFDNDSLCSVDLIPVDIEF,TTYEKFDNDSLCSVDLIPVDIEFW
pfam,IFSTVSKLHNQTANNLSILVLSLN,FSTVSKLHNQTANNLSILVLSLNG,STVSKLHNQTANNLSILVLSLNGI


>E3LLJ4_CAERE/4-302 E3LLJ4.1 PF10318.12;7TM_GPCR_Srh;
before match: ETPNFVSNSLHLISIITTPIHLIGFYCILFKTPESMKSVKWGMFHVHFWCTLMDWSLTVITIPYLLSPVAAGVPLGFANVLGISTDFQCYFALTSVAAQGMAFVLIFENRYFLIFARNTSWRYIRVVFIIINYCAVFCVFIPLLTMIPEQTEARKAVLKILPDLPEALDVKLIFVLSTDISYILISAVFMESFLSTEAAIFVVLLWTNFKLTRSAQHSLKTMKLQKKFLLAMYIQAAVMFFNLVIPVSYF

matching: IFSTVSKLHNQTANNLSILVLSLNGI

after match: ASTTIMLWVHKPFREACFDLFRC
Match range (1-based): 251-276
-
>BHF
before match: MVHDTEQLLAQGHHEEETECGKYGKLPKKGSECKKHGILCRILTALHLKKRRTKHDHQKLLSESQEHIDASTNKTKKKAKKDKRKNKPPKKDSETSKPAQTTISRLPSNRNNNNANS

matching: FATTYEKFDNDSLCSVDLIPVDIEFW

after match: DMENEPVDQLPHEIXESVHMYGDDRFGERLIDRAQNKYAPLDEKQRSESHGAGEYLKHQWKGQGAKKARKVRSNGV
Match range (1-based): 118-143


In [82]:
df['name'].values

array(['A0A6Q2YJA0_ESOLU/18-823 A0A6Q2YJA0.1 PF04147.15;Nop14;',
       'A0A022QCL9_ERYGU/105-368 A0A022QCL9.1 PF03936.19;Terpene_synth_C;',
       'A0A098M3F2_9BACL/57-334 A0A098M3F2.1 PF01297.20;ZnuA;',
       'A0A1Y2CQP8_9FUNG/32-76 A0A1Y2CQP8.1 PF00187.22;Chitin_bind_1;',
       'B4HZG1_DROSE/1-124 B4HZG1.1 PF06382.14;Protamine_like;',
       'E3LLJ4_CAERE/4-302 E3LLJ4.1 PF10318.12;7TM_GPCR_Srh;'],
      dtype=object)

In [70]:
%debug

> /Users/olgabot/opt/miniconda3/envs/botryllus/lib/python3.10/site-packages/pandas/core/internals/blocks.py(1979)check_ndim()
   1977             )
   1978         if len(placement) != len(values):
-> 1979             raise ValueError(
   1980                 f"Wrong number of items passed {len(values)}, "
   1981                 f"placement implies {len(placement)}"



ipdb>  u


> /Users/olgabot/opt/miniconda3/envs/botryllus/lib/python3.10/site-packages/pandas/core/internals/blocks.py(1937)new_block()
   1935 
   1936     values, _ = extract_pandas_array(values, None, ndim)
-> 1937     check_ndim(values, placement, ndim)
   1938 
   1939     if klass is None:



ipdb>  u


> /Users/olgabot/opt/miniconda3/envs/botryllus/lib/python3.10/site-packages/pandas/core/internals/managers.py(1162)insert()
   1160             value = ensure_block_shape(value, ndim=self.ndim)
   1161 
-> 1162         block = new_block(values=value, ndim=self.ndim, placement=slice(loc, loc + 1))
   1163 
   1164         for blkno, count in _fast_count_smallints(self.blknos[loc:]):



...

ipdb>  u


> /var/folders/7z/r1593ybs1sj2ks5zzl9vy8840000gn/T/ipykernel_58253/836512331.py(4)<module>()
      2     print(f'--- {isoform} ---')
      3     for i, row in df.iterrows():
----> 4         pivoted = kmer_utils.subset_gene_kmers(
      5             bhf_pfam_kmers, col="name__pfam", gene_symbol=row.name, other="pfam"
      6         )



ipdb>  row['name']


'A0A6Q2YJA0_ESOLU/18-823 A0A6Q2YJA0.1 PF04147.15;Nop14;'


ipdb>  q


In [69]:
bhf_pfam_kmers.head()

,pfam_name,kmer__pfam,kmer_hp,0,1,hashval,i__pfam,name__pfam,pfam_id,pfam_id_full,i__botryllus,kmer__botryllus,name__botryllus
0,2-Hacid_dh,VEGVQWVNTLTDHNDKAALNTLLE,hphhphhpphpppppphhhpphhp,F3L153_9GAMM/5-309,F3L153.1,6861319682135420868,99,F3L153_9GAMM/5-309 F3L153.1 PF00389.33;2-Hacid...,PF00389,PF00389.33,20,GKYGKLPKKGSECKKHGILCRILT,BHF_isoform1
1,2-Hacid_dh,VEGVQWVNTLTDHNDKAALNTLLE,hphhphhpphpppppphhhpphhp,F3L153_9GAMM/5-309,F3L153.1,6861319682135420868,99,F3L153_9GAMM/5-309 F3L153.1 PF00389.33;2-Hacid...,PF00389,PF00389.33,20,GKYGKLPKKGSECKKHGILCRILT,BHF_isoform2
2,2-Hacid_dh_C,VEGVQWVNTLTDHNDKAALNTLLE,hphhphhpphpppppphhhpphhp,F3L153_9GAMM/98-277,F3L153.1,6861319682135420868,6,F3L153_9GAMM/98-277 F3L153.1 PF02826.22;2-Haci...,PF02826,PF02826.22,20,GKYGKLPKKGSECKKHGILCRILT,BHF_isoform1
3,2-Hacid_dh_C,VEGVQWVNTLTDHNDKAALNTLLE,hphhphhpphpppppphhhpphhp,F3L153_9GAMM/98-277,F3L153.1,6861319682135420868,6,F3L153_9GAMM/98-277 F3L153.1 PF02826.22;2-Haci...,PF02826,PF02826.22,20,GKYGKLPKKGSECKKHGILCRILT,BHF_isoform2
4,AAA_10,INPLSFIDSINETNRKLGISKFVD,hphhphhpphpppppphhhpphhp,F4L4C0_HALH1/172-500,F4L4C0.1,6861319682135420868,80,F4L4C0_HALH1/172-500 F4L4C0.1 PF12846.10;AAA_10;,PF12846,PF12846.10,20,GKYGKLPKKGSECKKHGILCRILT,BHF_isoform1
